In [34]:
# !pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup 

Вам необходимо написать функцию, 
которая будет основана на поиске по сайту habr.com. 
Функция в качестве параметра должна принимать 
список запросов для поиска 
(например, ['python', 'анализ данных'])

и на основе материалов, 
попавших в результаты поиска по каждому запросу, 
возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа 
только с одной (первой) страницей результатов 
поисковой выдачи для каждого запроса. 
Материалы в датафрейме не должны дублироваться, 
если они попадали в результаты поиска для нескольких запросов из списка.

In [35]:
# вариант 1
# выдает где в статье встречается весь список
base_url = 'https://habr.com'
def search_article(list_param):
  url_search = base_url + '/ru/search/'
  resp = requests.get(url_search, params = 
                      dict(q = list_param,
                           target_type='posts',
                           order='relevance'))
  soup = BeautifulSoup(resp.text)
  data = []
  for article_tag in soup.find_all('article'):
    dt = article_tag.find('time').get('datetime')
    h2_tag = article_tag.find('h2')
    title = h2_tag.get_text()
    link = h2_tag.find('a').get('href')
    link = base_url + link 
    # rating = article_tag.find('span', class_='tm-votes-meter__value').get_text()
    data.append([dt, title, link])
  df = pd.DataFrame(data,columns=['date', 'title','link'])
  df.drop_duplicates(inplace=True)
  df.date = pd.to_datetime(df.date).dt.date
  # print(len(df)) == 20
  return df
  

pd.set_option('display.max_colwidth', None)
search_article(['python', 'анализ данных']).head() 

,date,title,link
0,2021-12-10,Интенсив для повышения квалификации: как использовать Python для анализа данных,https://habr.com/ru/company/netologyru/news/t/594683/
1,2017-06-27,Использование Python и Excel для обработки и анализа данных. Часть 1: импорт данных и настройка среды,https://habr.com/ru/company/otus/blog/331746/
2,2017-06-30,Использование Python и Excel для обработки и анализа данных. Часть 2: библиотеки для работы с данными,https://habr.com/ru/company/otus/blog/331998/
3,2022-06-09,Шаблон новичка на пути PANDAS в искусстве анализа данных,https://habr.com/ru/post/670668/
4,2021-05-06,"Python, исследование данных и выборы: часть 3",https://habr.com/ru/post/556042/


In [36]:
# вариант 2
# выдает где в статье встречаются слова из списка - по отдельности для каждого из списка параметров функции

# если все-таки запросы надо смотреть по очереди 
# (для исключения дубликатов)

base_url = 'https://habr.com'
def search_article(list_param):
  data = []
  for one_search in list_param:
    url_search = base_url + '/ru/search/'
    resp = requests.get(url_search, params = 
                      dict(q = one_search,
                           target_type='posts',
                           order='relevance'))
    soup = BeautifulSoup(resp.text)
    for article_tag in soup.find_all('article'):
      dt = article_tag.find('time').get('datetime')
      h2_tag = article_tag.find('h2')
      title = h2_tag.get_text()
      # ошибка при поиске "анализа данных"
      # AttributeError: 'NoneType' object has no attribute 'get'
      try:
        link = h2_tag.find('a').get('href')
        # print(link)
      except AttributeError:
        link = 'ошибка кодировки'   
      link = base_url + link
      
    # rating = article_tag.find('span', class_='tm-votes-meter__value').get_text()
      data.append([dt, title, link])
  # ------------------------------------------------------
  df = pd.DataFrame(data,columns=['date', 'title','link'])
  df.drop_duplicates(inplace=True)
  df.date = pd.to_datetime(df.date).dt.date
  return df
  

pd.set_option('display.max_colwidth', None)
res = search_article(['python'])
# print(len(res)) == 20
res = search_article(['python', ' анализ данных'])
# print(len(res)) == 40 что логично по 20 статей для каждого, дубликатов не было, вероятно 
res.head()


,date,title,link
0,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31 января,https://habr.com/ru/company/southbridge/news/t/646825/
1,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Python 2.x",https://habr.com/ru/news/t/498364/
2,2021-12-13,"Жаждущим автоматизации: открытый урок «ChatOps c Errbot на Python», 21 декабря",https://habr.com/ru/company/southbridge/news/t/595093/
3,2021-07-06,Python Community Meetup 8/07: видео и материалы встречи,https://habr.com/ru/company/raiffeisenbank/news/t/566370/
4,2022-01-13,"Открытый урок «Пишем Custom Prometheus Exporter на Python», 19 января",https://habr.com/ru/company/southbridge/news/t/645485/


Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов; в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков: <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [54]:
#для меньшей путанности работать буду с вариатном №1
base_url = 'https://habr.com'
def search_article_page(list_param, page):
  data =[]
  for num_page in range(int(page) + 1):
    url_search = base_url + '/ru/search/' +f'page{num_page}'
    resp = requests.get(url_search, params = 
                      dict(q = list_param,
                           target_type='posts',
                           order='relevance'))
    soup = BeautifulSoup(resp.text)
    for article_tag in soup.find_all('article'):
      dt = article_tag.find('time').get('datetime')
      h2_tag = article_tag.find('h2')
      title = h2_tag.get_text()
      link = h2_tag.find('a').get('href')
      link = base_url + link

      resp_text = requests.get(link)
      b_text = BeautifulSoup(resp_text.text)
      try:
        text_b = b_text.find('div', id='post-content-body')
        text_one = text_b.get_text()
      except AttributeError:
        text_one = 'ошибка'
      rating = article_tag.find('span', class_='tm-votes-meter__value').get_text()
      data.append([dt, title, link,text_one,rating])
    df = pd.DataFrame(data,columns=['date', 'title','link','text','likes'])
    df.drop_duplicates(inplace=True)
    df.date = pd.to_datetime(df.date).dt.date
  return df
  

# pd.set_option('display.max_colwidth', None)
res = search_article_page(['python', 'анализ данных'], 1)
res.head(2) 

date  \
0  2021-12-10   
1  2017-06-27   

                                                                                                   title  \
0                        Интенсив для повышения квалификации: как использовать Python для анализа данных   
1  Использование Python и Excel для обработки и анализа данных. Часть 1: импорт данных и настройка среды   

                                                    link  \
0  https://habr.com/ru/company/netologyru/news/t/594683/   
1          https://habr.com/ru/company/otus/blog/331746/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           